In [5]:
import pandas as pd
import numpy as np
import matplotlib as plt
loans = pd.read_csv("LOAN-train.csv")

In [11]:
def rowCount():
    return len(loans)

print(rowCount())

614


In [ ]:
loans.head(6) # check first 6 lines of data

In [ ]:
loans.describe() # numerical data

In [ ]:
print(loans['Property_Area'].head(7)) # looking at a specific column of the data

In [ ]:
loans['Property_Area'].value_counts() # count occurances of each of the three values

In [ ]:
loans['ApplicantIncome'].hist(bins=50)

In [ ]:
loans.boxplot(column='ApplicantIncome')

In [ ]:
loans.boxplot(column='ApplicantIncome', by='Education')

In [ ]:
loans['LoanAmount'].hist(bins=50)

In [ ]:
loans.boxplot(column='LoanAmount')

In [ ]:
print('Frequency Rable for Credit History')
temp1 = loans['Credit_History'].value_counts(ascending=True)
print(temp1)

In [ ]:
print("\nProbability of getting a loan for each Credit History class: ")
temp2 = loans.pivot_table(values='Loan_Status',index=['Credit_History'],aggfunc=lambda x: x.map({'Y':1,'N':0}).mean())
print(temp2)

In [ ]:
# Adding gender

print("\nProbability of getting a loan for each Credit History class BY GENDER: ")
temp2gender = loans.pivot_table(values='Loan_Status',index=['Credit_History', 'Gender'],aggfunc=lambda x: x.map({'Y':1,'N':0}).mean())
print(temp2gender)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(12,3))
ax1 = fig.add_subplot(121)
ax1.set_xlabel('Credit_History')
ax1.set_ylabel('Count of Applicants')
ax1.set_title("Applicants by Credit History")
temp1.plot(kind='bar')

ax2 = fig.add_subplot(122)
temp2.plot(kind='bar')
ax2.set_xlabel('Credit_History')
ax2.set_ylabel('Probability of getting a loan')
ax2.set_title("Probability of getting a loan by credit history")

fig2 = plt.figure(2)
fig2 = plt.figure(figsize=(12,3))
ax3 = fig2.add_subplot(223)
temp2gender.plot(kind='bar', color=['red','blue'])
ax3.set_xlabel('Credit_History by Gender')
ax3.set_ylabel('Probability of getting a loan')
ax3.set_title('Probability of getting a loan by credit history and gender')

In [ ]:
N=2
width = 0.35
ind = np.arange(N)
#Data
alldata = loans.pivot_table(values='Loan_Status',index=['Gender','Credit_History'],aggfunc=lambda x: x.map({'Y':1,'N':0}).mean())
print(alldata)
mendata = alldata['Male']
print("\nMen data")
print(mendata)
womendata = alldata['Female']
print("\nWomen data")
print(womendata)

In [ ]:
# Creating a stacked bar chart to compare men and women probabilities
p1 = plt.bar(ind,mendata,width,color='#d62728')
p2 = plt.bar(ind,womendata,width)
plt.xlabel("Credit_History")
plt.ylabel("Probability of Loan")
plt.title("Probability of getting loan on credit history and gender")
plt.legend((p1[0],p2[0]), ('Men', 'Women'))
plt.xticks(ind,('0','1'))
plt.show

In [ ]:
temp3 = pd.crosstab(loans['Credit_History'], loans['Loan_Status'])
temp3.plot(kind='bar',stacked=True,color=['red','blue'],grid=False) 

In [ ]:
t3 = pd.crosstab([loans['Credit_History'],loans['Gender']],loans['Loan_Status'])
t3.plot(kind='bar',stacked=True,color=['red','blue'],grid=False)

In [ ]:
loans

In [ ]:
# Create a new function
def num_missing(x):
    return(sum(x.isnull()))

# Apply per column
print("missing values per column:")
print(loans.apply(num_missing, axis=0))

# Apply per row:
print("\nMissing values per row:")
print(loans.apply(num_missing, axis=1).head())

In [ ]:
loans['LoanAmount'].fillna(loans['LoanAmount'].mean(),inplace=True) # Fill missing LoanAmounts with the mean of known amounts
loans.head(1) # This first value initially had 'NaN' but now contains a value

In [ ]:
from scipy.stats import mode
print(loans['Gender'].mode()[0])

In [ ]:
# Fill in 'NaN' values for genders with most common gender
loans['Gender'].fillna(loans['Gender'].mode()[0], inplace=True)

In [ ]:
# Fill in 'NaN' values for marriges with most common marrige status
loans['Married'].fillna(loans['Married'].mode()[0], inplace=True)
print(loans['Married'].mode()[0])

In [ ]:
# Fill in 'NaN' values for self employeed with most common self employeed status
loans['Self_Employed'].fillna(loans['Self_Employed'].mode()[0], inplace=True)
print(loans['Self_Employed'].mode()[0])

We filled in the LoanAmount based on the median, a simple calculation.
A better one would be based on the median for similar Self_Employeed and Education.

In [ ]:
table = loans.pivot_table(values='LoanAmount', index='Self_Employed', columns='Education', aggfunc=np.median)
print(table)
# Values of this table are loan amoounts indexed by whether or not they're self employeed and a graduate or not.

In [ ]:
def fage(x):
    return table.loc(x['Self_Employed'],x['Education'])

In [ ]:
tableCH_SExEdu = loans.pivot_table(values='Credit_History', index='Self_Employed', columns='Education', aggfunc=np.median)
print(tableCH_SExEdu)
# It looks like all of the midean values for credit histories is 1

In [ ]:
def ffch(x):
    return tableCH_SExEdu.loc(x['Self_Employed'], x['Education'])

#loans['Credit_History'].fillna(loans[loans['Credit_History'].isnull()].apply(ffch,axis=1), inplace=True)

In [ ]:
t2 = pd.crosstab([loans['Self_Employed'], loans['Education']], loans['Dependents'])
print(t2)

In [ ]:
def coding(col, codeDict):
    colCoded = pd.Series(col, copy=True)
    for key, value in codeDict.items():
        colCoded.replace(key, value, inplace=True)
    return colCoded

loans['Dependents_Encoded'] = coding(loans['Dependents'], {'0':0, '1':1, '2':2, '3+':3})

In [ ]:
#loans['Dependents_Encoded'] will now replace all 3+ with just 3

print(loans['Dependents_Encoded'].median()) # Runs without error

In [ ]:
t3 = loans.pivot_table(values='Dependents_Encoded', index='Self_Employed', columns='Education', aggfunc=np.median)
print(t3)

In [ ]:
def fixDependents(x):
    return t3.loc[x['Self_Employed'], x['Education']]

testRow = loans.iloc[598]
print(fixDependents(testRow))